# Data Preparation

From ISTAT data web site search for the population indicators and set:
- age to total
- martial status to total
- period last 5 years
- territory all

Export the data in both Excel and CSV format beacuse in CSV file the relationship of State - Geographic Area - Macro Region - Region - Province - Comune is lost so we have to rebuild it

Excel file: **ab69eff1-7aaa-4f13-9874-d07c9b7db9ad.xls**

CSV file: **DCIS_POPRES1_15032020152523718.csv**

## Location information maniputation

From the excel file **ab69eff1-7aaa-4f13-9874-d07c9b7db9ad.xls** copy the data from the first column (only the names of the regional entities) into a **location_step_0.txt** file

In [9]:
import os
import sys


INPUT_FILE_NAME = 'location_step_0.txt'
STEP_1_FILE_NAME = 'location_step_1.txt'
STEP_2_FILE_NAME = 'location_step_2.csv'
SEPARATOR = '*'
STATE = 'State'
GEOGRAPHIC_AREA = 'Geographic Area'
MACRO_REGION = 'Macro Region'
REGION = 'Region'
PROVINCE = 'Province'
COMUNE = 'Comune'

first_line_flag = False
geographic_area_flag = False
current_directory = os.getcwd()
#print(current_directory)

with open(STEP_1_FILE_NAME, 'w', encoding = 'UTF-8') as output_file:
    with open(INPUT_FILE_NAME, encoding = 'UTF-8') as input_file:
        lines = input_file.readlines()
        for line in lines:
            line = line.replace('\n','')
            if first_line_flag == False:
                first_line_flag = True
                line = STATE + SEPARATOR + line
            elif '        ' in line:
                line = line.replace('        ', COMUNE + SEPARATOR)
                if geographic_area_flag == True:
                    geographic_area_flag = False
            elif '      ' in line:
                line = line.replace('      ', PROVINCE + SEPARATOR)
            elif '    ' in line:
                line = line.replace('    ', REGION + SEPARATOR)
            elif '  ' in line:
                if geographic_area_flag == False:
                    geographic_area_flag = True
                    line = line.replace('  ', GEOGRAPHIC_AREA + SEPARATOR)
                else:
                    line = line.replace('  ', MACRO_REGION + SEPARATOR)

            output_file.write("{};\n".format(line))

# Fix need
# - Change 'Geographic Area*Nord-est;' to 'Macro Region*Nord-est;'
# - Change 'Region*Provincia Autonoma Bolzano / Bozen;' to 'Province*Provincia Autonoma Bolzano / Bozen;'
# - Delete Province*Bolzano / Bozen;
# - Change 'Region*Provincia Autonoma Trento;' to 'Province*Provincia Autonoma Trento;'
# - Delete Province*Trento;
# - Add 'Macro Region*Centro;'
# - Change 'Geographic Area*Isole;' to 'Macro Region*Isole;'



with open('temp' + STEP_1_FILE_NAME, 'w', encoding = 'UTF-8') as output_file:
    with open(STEP_1_FILE_NAME, encoding = 'UTF-8') as input_file:
        lines = input_file.readlines()
        for line in lines:
            if 'Geographic Area*Nord-est;' in line:
                line = 'Macro Region*Nord-est;\n'
            if 'Region*Provincia Autonoma Bolzano / Bozen;' in line:
                line = 'Province*Provincia Autonoma Bolzano / Bozen;\n'
            if 'Province*Bolzano / Bozen;' in line:
                line = ''
            if 'Region*Provincia Autonoma Trento;' in line:
                line = 'Province*Provincia Autonoma Trento;\n'
            if 'Province*Trento;' in line:
                line = ''
            if 'Geographic Area*Centro;' in line:
                line = 'Geographic Area*Centro;\nMacro Region*Centro;\n'
            if 'Geographic Area*Isole;' in line:
                line = 'Macro Region*Isole;\n'

            output_file.write("{}".format(line))

os.remove(STEP_1_FILE_NAME)

#print(sys.platform)

if 'linux' in sys.platform: # linux
    folder_delimiter = '/'
elif 'darwin' in sys.platform: # OS X
    folder_delimiter = '/'
elif 'win' in sys.platform: # Windows
   folder_delimiter = '\\'

os.rename(current_directory + folder_delimiter + 'temp' + STEP_1_FILE_NAME, current_directory + folder_delimiter + STEP_1_FILE_NAME)


Now let's build the hierarchical relationship between State - Geographic Area - Macro Region - Region - Province - Comune

In [10]:
with open(STEP_2_FILE_NAME, 'w', encoding = 'UTF-8') as output_file:
    output_file.write("{},{},{},{},{},{};\n".format(STATE, GEOGRAPHIC_AREA, MACRO_REGION, REGION, PROVINCE, COMUNE))
    with open(STEP_1_FILE_NAME, encoding = 'UTF-8') as input_file:
        lines = input_file.readlines()
        for line in lines:
            line = line.replace('\n','').replace(';','')
            if STATE in line:
                categoria, state = line.split(SEPARATOR)
            if GEOGRAPHIC_AREA in line:
                categoria, geographic_area = line.split(SEPARATOR)
            if MACRO_REGION in line:
                categoria, macro_region = line.split(SEPARATOR)
            if REGION in line:
                categoria, region = line.split(SEPARATOR)
            if PROVINCE in line:
                categoria, province = line.split(SEPARATOR)
            if COMUNE in line:
                categoria, comune = line.split(SEPARATOR)
                output_file.write("{},{},{},{},{},{};\n".format(state, geographic_area, macro_region, region, province, comune))


Now let's model the data with Power BI